# Noisy H and CZ

In [1]:
from qiskit import QuantumCircuit
import numpy as np
from qiskit_aer import AerSimulator
from qiskit_aer.noise import NoiseModel, depolarizing_error, ReadoutError
from qiskit.visualization import plot_histogram 
import typing
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister

from qiskit_aer import Aer
from qiskit import transpile
from qiskit_aer.primitives import Sampler
from qiskit.primitives.backend_sampler import BackendSampler
from qiskit.quantum_info import Statevector
from qiskit.circuit import Instruction
from matplotlib import pyplot as plt
from qiskit.quantum_info import state_fidelity

In [3]:
import steane_encoding_module as sem

In [7]:
logical_cz = sem.create_cz_block(8, True)


8
8
8
8
8
8
8
8


In [8]:
logical_h = sem.create_h_block(8, True)

In [9]:
print(logical_cz)

Instruction(name='cz_block', num_qubits=16, num_clbits=16, params=[])


In [10]:
# 5% chance of flipping 0 <-> 1
readout_err = ReadoutError([[0.98, 0.02],  # P(measured 0 | actual 0), P(1 | 0)
                            [0.02, 0.98]]) # P(0 | 1), P(1 | 1)

noise_model = NoiseModel()

noise_model.add_all_qubit_quantum_error(depolarizing_error(0.01,1), ['id'])
noise_model.add_all_qubit_quantum_error(depolarizing_error(0.01,1), ['h'])
noise_model.add_all_qubit_quantum_error(depolarizing_error(0.01,2),['cz'])
noise_model.add_all_qubit_quantum_error(depolarizing_error(0.01,2),['cx'])

In [ ]:
depol_probs = [0.0, 0.02, 0.04, 0.06, 0.08, 0.1, 0.12, 0.14, 0.16, 0.18, 0.2] 
fidelity_results = {}

for p in depol_probs:
    # New noise model for each depolarizing probability
    noise_model = NoiseModel()
    qc_ideal = sem.SteaneBlock(8,8,True)
    qc_noise = sem.SteaneBlock(8,0,False)

    qc_ideal.append(sem.create_id_block(8, True), range(0,8), range(0,8))
    qc_ideal.append(logical_h, range(0,8), range(0,8))

    # Apply depolarizing error with probability p
    noise_model.add_all_qubit_quantum_error(depolarizing_error(p, 1), ['id', 'h'])
    noise_model.add_all_qubit_quantum_error(depolarizing_error(p*5, 2), ['cz'])

    qc_noise.append(sem.create_id_block(8, False), range(0,8), range(0,8))
    qc_noise.append(logical_h, range(0,8), range(0,8))


    qc_noise.save_statevector(label='state_post', pershot=True, conditional=True)

    # Run simulation
    backend_noise = AerSimulator(noise_model=noise_model)
    transpiled_noise = transpile(qc_noise, backend_noise)
    job = backend_noise.run(transpiled_noise, shots=200000)
    result = job.result()

    